<p><img src="https://files.cdn.thinkific.com/file_uploads/133292/images/a9e/c1a/bc3/P02.jpg"></p>

`MCSDSC02P01V3______`

`MCSDSC02P01V2______`

`MCSDSELIMC02P01V1010621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Как обычно, заголовки (комментариев) будут трех цветов. <font color="89DD00">Зеленый</font> — все правильно (и если комментария нет, то это также значит, что все правильно). <font color="FFB800">Желтый</font> — можно доработать. <font color="FF0F00">Красный</font> — нужно исправить.

## Постановка задачи

Необходимо проанализировать информацию, представленную менеджером проекта о тестовом запуске приложения "Без очереди", и ответить на следующие вопросы:
* На какие кафе приходится больше всего заказов? А на какие дни?
* Чаще ли приложением пользуются именно в обеденное время?
* Насколько большими компаниями  (или наоборот поодиночке) приходят в кафе?

## Импорт данных и библиотек

In [27]:
import pandas as pd

Прочитаем файл `bez_ocheredi.csv` и сохраним его в переменную `bo_data`.

In [333]:
bo_data=pd.read_csv("../../data/bez_ocheredi.csv")

Получим первые 10 строк таблицы.

In [334]:
bo_data.head(10)

,userID,ресторан,weekday,ordered,Столик для
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3
5,51e0962387f7,Там Ем,Friday,12:30,2
6,c4f260a2b48a,NaN,Saturday,15:00,6
7,6b71309d6a8a,Крафтейнер,Monday,10:30,1
8,c5b969cb63a2,Бар Сибирь,Wednesday,15:30,2
9,b1570962e68c,Бар Сибирь,Wednesday,12:30,3


Получим общую информацию о данных из таблицы.

In [335]:
bo_data.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userID        1024 non-null   object
 1    ресторан     959 non-null    object
 2    weekday      1024 non-null   object
 3    ordered      1024 non-null   object
 4    Столик для   1024 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 40.1+ KB


Столбцы в `bo_data` содержат следующую информацию:

* Идентификационный номер посетителя
* Название ресторана
* День недели
* Время заказа
* Количество лиц за столом

### Выводы

* Данные содержатся в формате таблицы состоящей из 5 граф и 1024 строк
* В графе "Restaraunts" содержатся пустые значения 

<font color="89DD00">Комментарий ассистента</font>

Тут все ОК!

## Предобработка данных

Получим названия столбцов.

In [336]:
bo_data.head(0)

,userID,ресторан,weekday,ordered,Столик для


<font color="FFB800">Комментарий ассистента</font>

Наименования столбцов лучше вызывать специальным методом `bo_data.columns`

Обращаем внимание, что помимо названий на разном языке, в них есть пробелы, которые затрудняют доступ к данным. Переименуем столбцы и проверим результат.

In [368]:
bo_data.set_axis(['UserID','Restaraunt','Weekday','Ordered','Sum'],axis='columns',inplace=True)

In [369]:
bo_data.head(0)

,UserID,Restaraunt,Weekday,Ordered,Sum


Еще раз проверим данные на наличие пропусков, просуммировав их.

In [410]:
bo_data.isnull().sum()

UserID        0
Restaraunt    0
Weekday       0
Ordered       0
Sum           0
dtype: int64

Менеджер проекта предупреждал, что некоторые заказы заводились на тестовую карточку кафе без названия, поэтому есть смысл заполнить такой пропуск словом admin.

In [341]:
bo_data=bo_data.fillna('admin')

Проверяем, что пропусков не осталось.

In [342]:
bo_data.isnull().sum()

UserID        0
Restaraunt    0
Weekday       0
Ordered       0
Table         0
dtype: int64

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [343]:
bo_data.duplicated().sum()

24

In [344]:
bo_data=bo_data.drop_duplicates().reset_index(drop=True)

In [345]:
bo_data.duplicated().sum()

0

<font color="89DD00">Комментарий ассистента</font>

С этим блоком тоже все в порядке.

Скорее всего, такие полные дубликаты говорят о сбое при выгрузке данных. Необходимо сообщить об этом.

Еще одна проблема, о которой предупреждал менеджер проекта — дубликаты карточки Iskra Bar с неправильными названием бара - "Искра" и "Искр". На них заводились реальные заказы, поэтому необходимо вычислить их и переименовать на правильное название — Iskra Bar.

Объявим функцию `replace_restaurant()`, которая заменяет неправильное название, а потом проверяет успешность выполнения замены подсчетом (если замена прошла успешно — результат подсчета должен быть 0).

In [346]:
bo_date_replace=[]
def replace_restaraunt(first_name, new_name):
    bo_data.replace(first_name,new_name,inplace=True)
    return bo_data[bo_data['Restaraunt']==first_name].count()

<font color="FFB800">Комментарий ассистента</font>

Почти ОК, но лучше при создании функции не использовать конкретное имя таблицы, чтобы ее можно было применять к разным объектам. Лучше функции передать три аргумента - имя таблицы и уже имеющиеся старое и новое название. Но в рамках этого задания функция работает в порядке!

Передаем функции названия 'Искра' и 'Искр'.

In [347]:
replace_restaraunt('Искр', 'Iskra Bar')

UserID        0
Restaraunt    0
Weekday       0
Ordered       0
Table         0
dtype: int64

In [348]:
replace_restaraunt('Искра', 'Iskra Bar')

UserID        0
Restaraunt    0
Weekday       0
Ordered       0
Table         0
dtype: int64

### Выводы

* При выполнении предварительной обработки данных, были изменены названия граф, а так же в соответствии с комментариями менеджера тестовые заказы были обозначены в графе "Restaraunt" словом "admin", и заказы, сделанные на дубликат карточек, были переименованы в графе "Resataraunt" из "Искр","Искра" в "Iskra bar", для дальнейшего анализа данных.

## Распределение заказов в течение недели

Считаем, на какие кафе приходится больше всего заказов.

In [349]:
bo_data['Restaraunt'].value_counts()

Iskra Bar     287
Там Ем        200
Бар Сибирь    159
Северия       107
Крафтейнер     78
Коворкафе      66
Азия           62
admin          41
Name: Restaraunt, dtype: int64

Группируем по `weekday` и считаем, на какие дни приходится больше всего заказов.

In [350]:
bo_data.groupby('Weekday').count()

,UserID,Restaraunt,Ordered,Table
Weekday,,,,
Friday,110,110,110,110
Monday,141,141,141,141
Saturday,52,52,52,52
Sunday,24,24,24,24
Thursday,180,180,180,180
Tuesday,251,251,251,251
Wednesday,242,242,242,242


<font color="FFB800">Комментарий ассистента</font>

Тоже почти супер. Но колонки с одинаковыми значениями мешают, они нам ни к чему. Также удобнее смотреть уже отсортированные данные, чтобы глазами не искать самое большое значение - например так `bo_data.groupby('weekday')['Restaurant'].count().sort_values(ascending = False)`

Объявляем функцию, которая возвращает количество заказов для ресторана в определенный день.

In [351]:
def restaraunt_days(day,bar):
    rest_days=bo_data[(bo_data['Restaraunt']==bar)&(bo_data['Weekday']==day)]
    return rest_days['Restaraunt'].count()

<font color="FFB800">Комментарий ассистента</font>

Функция работает, но лучше передавать ей также имя таблицы. Аналогичный комментарий ко всем функциям по нашей таблице.

Передадим ей заведения Iskra Bar и Там Ем — два флагманских места, по которым просили результаты прежде всего — и понедельник, среду и пятницу в качестве дня недели.

In [352]:
restaraunt_days('Monday','Iskra Bar')

48

In [353]:
restaraunt_days('Wednesday','Iskra Bar')

53

In [354]:
restaraunt_days('Friday','Iskra Bar')

29

In [355]:
restaraunt_days('Monday','Там Ем')

25

In [356]:
restaraunt_days('Wednesday','Там Ем')

52

In [357]:
restaraunt_days('Friday','Там Ем')

24

Сведем результат в таблицу.

In [358]:
isk_mon=restaraunt_days('Monday','Iskra Bar')
isk_wed=restaraunt_days('Wednesday','Iskra Bar')
isk_fri=restaraunt_days('Friday','Iskra Bar')
tam_mon=restaraunt_days('Monday','Там Ем')
tam_wed=restaraunt_days('Wednesday','Там Ем')
tam_fri=restaraunt_days('Friday','Там Ем')
data=[['Iskra Bar',isk_mon,isk_wed,isk_fri],['Там Ем',tam_mon,tam_wed,tam_fri]]
columns=['Название ресторана','Понедельник','Среда','Пятница']
result=pd.DataFrame(data=data,columns=columns)
result

,Название ресторана,Понедельник,Среда,Пятница
0,Iskra Bar,48,53,29
1,Там Ем,25,52,24


### Выводы


### На данном этапе можно сделать выводы:
* Из выбранных ресторанов "Iskra Bar" является наиболее популярным заведением, так как за выбранные периоды времени в нём было совершено большее количество заказов;
* Пик заказов по выбранным ресторанам приходится на среду.


<h3><font color="#89DD00">Комментарий ассистента</font></h3>

По этому блоку критических замечаний нет, выводы тоже в порядке.

## Распределение заказов в течение дня

Сравним количества заказов, сделанных в обеденное и вечернее время — с 12 до 16 и с 16 до 22. Для этого объявим функцию, которая принимает данные и 2 аргумента времени.

In [365]:
def order_time(start,finish):
    return bo_data[(bo_data['Ordered']>=start)&(bo_data['Ordered']<finish)].count()

<font color="FFB800">Комментарий ассистента</font>

Как видно ниже, при работе функция выдает несколько столбцов таблицы с одинаковыми значениями. Поэтому лучше вернуть значение по одному столбцу - к примеру добавить перед `count()` наименование столбца (любого, к примеру `['user_id'].count()`).

Передадим промежуток с 12:00 и 16:00

In [366]:
order_time('12:00','16:00')

UserID        543
Restaraunt    543
Weekday       543
Ordered       543
Table         543
dtype: int64

И с 16:00 до 22:00

In [367]:
order_time('16:00','22:00')

UserID        338
Restaraunt    338
Weekday       338
Ordered       338
Table         338
dtype: int64

### Выводы

* В обеденное время больше заказов, из этого следует вывод, что в основном заказывают стол офисные работники.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Всё верно, в обеденное время столиков бронируется больше

## Типичное количество человек за столиком

Выясним, для какого количества человек обычно заказывают столик в разное время на примере Iskra Bar и Там Ем. Для этого напишем функцию, которая принимает данные, название ресторана, 2 времени и считает моду.

In [405]:
def quan_tabl(start,finish,bar):
    sum_tabl=bo_data['Sum']
    return sum_tabl[(bo_data['Ordered']>=start)&(bo_data['Ordered']<finish)&(bo_data['Restaraunt']==bar)].mode()

Проверим функцию на двух ресторанах, а также дневном и вечернем промежутке времени.

In [406]:
quan_tabl('12:00','16:00','Iskra Bar')

0    2
dtype: int64

In [407]:
quan_tabl('16:00','22:00','Iskra Bar')

0    1
dtype: int64

In [422]:
quan_tabl('12:00','16:00','Там Ем')

0    2
dtype: int64

In [423]:
quan_tabl('16:00','22:00','Там Ем')

0    1
dtype: int64

### Выводы

In [415]:
def quan_tabl_all(start,finish):
    sum_tabl_all=bo_data['Sum']
    return sum_tabl_all[(bo_data['Ordered']>=start)&(bo_data['Ordered']<finish)].mode()

In [361]:
order_time('12:00','16:00')

UserID        487
Restaraunt    487
Weekday       487
Ordered       487
Table         487
dtype: int64

In [418]:
quan_tabl_all('12:00','16:00')

0    1
dtype: int64

In [419]:
quan_tabl_all('16:00','22:00')

0    1
dtype: int64

In [412]:
bo_data.count()

UserID        1000
Restaraunt    1000
Weekday       1000
Ordered       1000
Sum           1000
dtype: int64

## Общие выводы

Самыми популярными ресторанами являются "Iskra Bar", "Там Ем", "Бар Сибирь" их доля заказов составляет более 60%. В основном заказы делают в будние дни, в среду пик, но ближе к пятнице количество заказов сокращается более чем в 2 раза, а в выходные дни по сравнению с пиком более чем в 5,5 раз.
Приложение чаще используют в обеденное время, на него приходится более всех 50% заказов.
В рестораны в основном приходят по одному, и в обеденное и в вечернее время. Но в популярные рестораны в обеденное время больше заказов совершается компаниями по 2 человека.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Все результаты и твои выводы правильные, отлично!

По желанию можешь внести изменения в функции :)